In [1]:
import pickle
import wikipediaapi
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from bs4 import BeautifulSoup
import re
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

# 20 seconds to load
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

d:\FoodData_Central_csv_2023-04-20\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
with open('vitamins_info.pkl','rb') as file:
    vitamins_info = pickle.load(file)

In [3]:
vitamins_info.head(2)

,Vitamin,Disease,Age Group,Gender,links
0,vitamin-a,Night blindness,Children,Female,https://www.hsph.harvard.edu/nutritionsource/v...
1,vitamin-a,Xerophthalmia,Children,Female,https://www.hsph.harvard.edu/nutritionsource/v...


In [9]:
vitamins_info.Disease.str.len().max()

39

In [6]:
sorted(vitamins_info.Disease.unique())

['Alopecia',
 'Anemia',
 'Anencephaly',
 'Ariboflavinosis',
 'Ataxia',
 'Beriberi',
 "Bitot's spots",
 'Bleeding disorders',
 'Blindness',
 'Blood in urine or stool',
 'Bone pain',
 'Bright red tongue',
 'Brittle nails',
 'Bruising easily',
 'Burning feet syndrome',
 'Cardiovascular problems',
 'Cheilosis',
 'Cleft palate',
 'Cognitive changes',
 'Cognitive decline',
 'Confusion',
 'Conjunctivitis',
 'Delayed tooth formation',
 'Dementia',
 'Depression',
 'Dermatitis',
 'Developmental delay',
 'Diarrhea',
 'Difficulty walking',
 'Dry and scaly skin',
 'Dry skin',
 'Easy bruising',
 'Edema',
 'Enteritis',
 'Fatigue',
 'Fetal growth restriction',
 'Fractures',
 'Gastrointestinal bleeding',
 'Hallucinations',
 'Heart defects',
 'Heavy menstrual bleeding',
 'Hematoma',
 'Hemolytic Anemia',
 'Hemorrhagic disease of the newborn',
 'Homocysteine buildup',
 'Hyperkeratosis',
 'Hyperpigmentation',
 'Immune system impairment',
 'Impaired coordination',
 'Impaired growth',
 'Impaired immune funct

In [11]:
dis = input('Enter the Disease')
dis

'Alopecia'

In [12]:
vitamin = vitamins_info[vitamins_info.Disease == dis]['Vitamin'].iloc[0]
vitamin

'vitamin-b7'

In [13]:
import requests
from bs4 import BeautifulSoup

def extract_important_text(title):
    url = f'https://en.wikipedia.org/wiki/{title}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract the main content of the page
        main_content = soup.find('div', {'id': 'mw-content-text'})

        if main_content:
            important_text = ' '.join([p.get_text() for p in main_content.find_all('p')])
            return important_text
        else:
            return 'Error: Main content not found on the Wikipedia page.'
    else:
        return f'Error fetching the Wikipedia page. Status code: {response.status_code}'

# Example usage
disease_name = dis
important_text = extract_important_text(disease_name)
cleaned_text = re.sub(r'\[\d+\]', '', important_text)
print(cleaned_text)


 Hair loss, also known as alopecia or baldness, refers to a loss of hair from part of the head or body.  Typically at least the head is involved. The severity of hair loss can vary from a small area to the entire body. Inflammation or scarring is not usually present. Hair loss in some people causes psychological distress.
 Common types include male- or female-pattern hair loss, alopecia areata, and a thinning of hair known as telogen effluvium. The cause of male-pattern hair loss is a combination of genetics and male hormones; the cause of female pattern hair loss is unclear; the cause of alopecia areata is autoimmune; and the cause of telogen effluvium is typically a physically or psychologically stressful event. Telogen effluvium is very common following pregnancy.
 Less common causes of hair loss without inflammation or scarring include the pulling out of hair, certain medications including chemotherapy, HIV/AIDS, hypothyroidism, and malnutrition including iron deficiency. Causes o

In [14]:
causes = [i.strip() for i in cleaned_text.split('.')]

['Hair loss, also known as alopecia or baldness, refers to a loss of hair from part of the head or body',
 'Typically at least the head is involved',
 'The severity of hair loss can vary from a small area to the entire body',
 'Inflammation or scarring is not usually present',
 'Hair loss in some people causes psychological distress',
 'Common types include male- or female-pattern hair loss, alopecia areata, and a thinning of hair known as telogen effluvium',
 'The cause of male-pattern hair loss is a combination of genetics and male hormones; the cause of female pattern hair loss is unclear; the cause of alopecia areata is autoimmune; and the cause of telogen effluvium is typically a physically or psychologically stressful event',
 'Telogen effluvium is very common following pregnancy',
 'Less common causes of hair loss without inflammation or scarring include the pulling out of hair, certain medications including chemotherapy, HIV/AIDS, hypothyroidism, and malnutrition including iron

In [46]:
causes = []
for i in [i.strip() for i in cleaned_text.split('.')]:
    if i.__contains__('cause'):
        causes.append(i.replace('\n',''))

15


In [45]:
for i in causes:
    print(i)

Hair loss in some people causes psychological distress
The cause of male-pattern hair loss is a combination of genetics and male hormones; the cause of female pattern hair loss is unclear; the cause of alopecia areata is autoimmune; and the cause of telogen effluvium is typically a physically or psychologically stressful event
Less common causes of hair loss without inflammation or scarring include the pulling out of hair, certain medications including chemotherapy, HIV/AIDS, hypothyroidism, and malnutrition including iron deficiency
The degree and pattern of baldness varies, but its most common cause is androgenic hair loss, alopecia androgenetica, or alopecia seborrheica, with the last term primarily used in Europe
The most severe form of the condition, cystic acne, arises from the same hormonal imbalances that cause hair loss and is associated with dihydrotestosterone production
Hair loss induced by cancer chemotherapy has been reported to cause changes in self-concept and body imag

In [40]:
causes[8]

'Other causes of hair loss include: Genetic forms of localized autosomal recessive hypotrichosis include: Hair follicle growth occurs in cycles'

In [47]:
Treatment = []

for i in [i.strip() for i in cleaned_text.split('.')]:
    if i.__contains__('Treatment'):
        Treatment.append(i.replace('\n',''))

In [50]:
print(*Treatment,sep = '\n')

Treatment of pattern hair loss may simply involve accepting the condition, which can also include shaving one's head
Treatments for the various forms of hair loss have limited success


In [44]:
for i in []: print(i)

In [32]:
a = 'vitamin-a'
url = vitamins_info[vitamins_info.Vitamin == a]['links'].iloc[0]



'https://www.hsph.harvard.edu/nutritionsource/vitamin-a'

In [34]:
a = vitamin
#getting info from webage and storign html content in a variable(webage_content)
url = vitamins_info[vitamins_info.Vitamin == a]['links'].iloc[0]
response = requests.get(url)
if response.status_code == 200:
    webpage_content = response.text
    soup = BeautifulSoup(webpage_content, 'html.parser')


    #retrieving food to consume from the webpage content
    food_sources_heading = soup.find('h3', string='Food Sources')
    if food_sources_heading:
        ul_tag = food_sources_heading.find_next('ul')
        if ul_tag:
            list_items = ul_tag.find_all('li')
            food_items = [item.get_text(strip=True) for item in list_items]
        else:print("No unordered list found after 'Food Sources' heading.")
    else:print("Food Sources heading not found.")


    #retreiving Signs of Deficiency and Toxicity Text from webpage contents
    def get_deficiency_toxicity_text(soup, tags):
        for tag in tags:
            deficiency_toxicity_tag = soup.find('h3', string=tag)
            if deficiency_toxicity_tag:
                deficiency_toxicity_text = ''
                next_tag = deficiency_toxicity_tag.find_next_sibling()
                while next_tag and next_tag.name != 'h3':
                    if next_tag.name == 'p':
                        deficiency_toxicity_text += next_tag.get_text() + '\n'
                    elif next_tag.name == 'ul':
                        for li in next_tag.find_all('li'):
                            deficiency_toxicity_text += f"- {li.get_text()}\n"
                    next_tag = next_tag.find_next_sibling()
                print(f'Signs of Deficiency and Toxicity Text (tag "{tag}"):')
                return deficiency_toxicity_text
        print('No matching <h3> tag found for Signs of Deficiency and Toxicity')

    tags_to_search = ['Signs of Deficiency and Toxicity', 'Signs of Deficiency', 'Signs of Deficiency and Toxicity ']
    deficiency_toxicity_text = get_deficiency_toxicity_text(soup, tags_to_search)
    
else:
    print('Error fetching the webpage')

Food Sources heading not found.
Signs of Deficiency and Toxicity Text (tag "Signs of Deficiency and Toxicity"):


In [35]:
print(f'-----{a} deficiency diseases-----')
print(*vitamins_info[vitamins_info.Vitamin == a]['Disease'].values,sep='\n')

print(f"----Food Should comsume for gaining {a}----")
print(*food_items,sep = '\n')

print('----Signs of Deficiency and Toxicity Text:----')
print(deficiency_toxicity_text.strip())

# print('\n----Summary:----')
# print(final_summary)

-----vitamin-b12 deficiency diseases-----
Pernicious Anemia
Fatigue
Shortness of breath
Pale or jaundiced skin
Smooth tongue
Difficulty walking
Numbness and tingling in hands and feet
Cognitive changes
Memory loss
Depression
----Food Should comsume for gaining vitamin-b12----
Leafy green vegetables (kale, spinach, broccoli), orange and yellow vegetables (carrots,sweet potatoes, pumpkin and otherwinter squash, summer squash)
Tomatoes
Red bell pepper
Cantaloupe, mango
Beef liver
Fish oils
Milk
Eggs
Fortified foods
----Signs of Deficiency and Toxicity Text:----
Measuring vitamin B12 in the blood is actually not the best way to determine whether someone is deficient, as some people with a deficiency can show normal B12 blood levels. Blood levels of methylmalonic acid, a protein breakdown product, and homocysteine are better markers that capture actual vitamin B12 activity. These values increase with a vitamin B12 deficiency. It is estimated that up to 15% of the general population has a vi

In [20]:
import requests
from bs4 import BeautifulSoup

def extract_important_text(title):
    url = f'https://en.wikipedia.org/wiki/{title}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract the main content of the page
        main_content = soup.find('div', {'id': 'mw-content-text'})

        if main_content:
            important_text = ' '.join([p.get_text() for p in main_content.find_all('p')])
            return important_text
        else:
            return 'Error: Main content not found on the Wikipedia page.'
    else:
        return f'Error fetching the Wikipedia page. Status code: {response.status_code}'

# Example usage
disease_name = 'Xerophthalmia'
important_text = extract_important_text(disease_name)


In [21]:
print(*important_text.split('\n'),sep = '\n')

Xerophthalmia (from Ancient Greek xērós (ξηρός) meaning "dry" and ophthalmos (οφθαλμός) meaning "eye") is a medical condition in which the eye fails to produce tears. It may be caused by vitamin A deficiency,[1] which is sometimes used to describe that condition, although there may be other causes.
 Xerophthalmia caused by a severe vitamin A deficiency is described by pathologic dryness of the conjunctiva and cornea. The conjunctiva becomes dry, thick and wrinkled. The first symptom is poor vision at night. If untreated, xerophthalmia can lead to dry eye syndrome, corneal ulceration, and ultimately to blindness as a result of corneal and retinal damage.
 Xerophthalmia usually implies a destructive dryness of the conjunctival epithelium due to dietary vitamin A deficiency—a rare condition in developed countries, but still causing much damage in developing countries. Other forms of dry eye are associated with aging, poor lid closure, scarring from a previous injury, or autoimmune disease

In [22]:
for i in important_text.split('\n'):
    if i.__contains__('cause'):
        print(i)

Xerophthalmia (from Ancient Greek xērós (ξηρός) meaning "dry" and ophthalmos (οφθαλμός) meaning "eye") is a medical condition in which the eye fails to produce tears. It may be caused by vitamin A deficiency,[1] which is sometimes used to describe that condition, although there may be other causes.
 Xerophthalmia caused by a severe vitamin A deficiency is described by pathologic dryness of the conjunctiva and cornea. The conjunctiva becomes dry, thick and wrinkled. The first symptom is poor vision at night. If untreated, xerophthalmia can lead to dry eye syndrome, corneal ulceration, and ultimately to blindness as a result of corneal and retinal damage.
 Xerophthalmia usually implies a destructive dryness of the conjunctival epithelium due to dietary vitamin A deficiency—a rare condition in developed countries, but still causing much damage in developing countries. Other forms of dry eye are associated with aging, poor lid closure, scarring from a previous injury, or autoimmune disease

In [23]:
for i in important_text.split('\n'):
    if i.__contains__('Treatment'):
        print(i)

 Treatment can occur in two ways: treating symptoms and treating the deficiency. Treatment of symptoms usually includes the use of artificial tears in the form of eye drops, increasing the humidity of the environment with humidifiers, and wearing wraparound glasses when outdoors. Treatment of the deficiency can be accomplished with a vitamin A or multivitamin supplement or by eating foods rich in vitamin A. Treatment with supplements and/or diet can be successful until the disease progresses as far as corneal ulceration, at which point only an extreme surgery can offer a chance of returning sight.


In [29]:
def split_text_into_chunks(text, chunk_size):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

def summarize_text(text, model, tokenizer):
    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt", max_length=1024)
    summary = model.generate(**tokens)
    return tokenizer.decode(summary[0], skip_special_tokens=True)

# Split the text into chunks of 1000 words (adjust as needed)
chunk_size = 500
text_chunks = split_text_into_chunks(important_text, chunk_size)

        # Summarize each chunk
summaries = [summarize_text(chunk, model, tokenizer) for chunk in text_chunks]

        # Concatenate the summaries
final_summary = ' '.join(summaries)

print('\nSummary:')
print(final_summary)

KeyboardInterrupt: 

In [30]:
def extract_important_text_wiki(title):
    url = f'https://en.wikipedia.org/wiki/{title}'
    headers = {'User-Agent': 'MyWikiScraper/1.0 (Eswar0210)'}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        important_text = response.text
        return important_text
    else:
        return f'Error fetching the Wikipedia page. Status code: {response.status_code}'

# Example usage
disease_name = 'Night blindness'
important_text = extract_important_text_wiki(disease_name)

In [31]:
print(important_text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-night-mode-clientpref-0 vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Nyctalopia - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-fea

In [ ]:
# Initialize Pegasus model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

def summarize_text(text, model, tokenizer):
    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt", max_length=1024)
    summary = model.generate(**tokens)
    return tokenizer.decode(summary[0], skip_special_tokens=True)

# Split the text into chunks of 500 words (adjust as needed)
chunk_size = 1000
text_chunks = split_text_into_chunks(important_text, chunk_size)

# Summarize each chunk
summaries = [summarize_text(chunk, model, tokenizer) for chunk in text_chunks]

# Concatenate the summaries
final_summary = ' '.join(summaries)

print('\nSummary:')
print(final_summary)